In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import metrics
#https://www.jianshu.com/p/ee91d8880bbd
#https://www.jianshu.com/p/a8d75f06194b

In [21]:
titanic = pd.read_csv(r'/Users/lano/DeepLearning/Kaggle/Titanic/train.csv')
test_data = pd.read_csv(r'/Users/lano/DeepLearning/Kaggle/Titanic/test.csv')

In [22]:
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())

In [23]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [24]:
titanic.loc[titanic["Sex"] == "male","Sex"] = 0
titanic.loc[titanic["Sex"] == "female","Sex"] = 1

In [25]:
titanic["Embarked"] = titanic["Embarked"].fillna("S")

In [26]:
titanic.loc[titanic["Embarked"] == "S","Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C","Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q","Embarked"] = 2

In [27]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0


In [28]:
#这里，人为的先选取部分有用特征
predictors = ["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]
#将线性回归方法导进来
alg = LinearRegression()
#将m个样本分成三份 n_folds 代表的是交叉验证划分几层（几份）
kf = KFold(titanic.shape[0],n_folds=3,random_state=1)#该行代码做的是交叉验证

predictions = []
for train,test in kf:
    #将训练数据拿出来，对原始数据取到建立好的特征 然后取出用于训练的那一部分
    train_predictors = (titanic[predictors].iloc[train,:])
    #print(train_predictors)

    #获取到数据集中交叉分类好的标签，即是否活了下来
    train_target = titanic["Survived"].iloc[train]
    #print(train_target)

    #将数据放进去做训练
    alg.fit(train_predictors,train_target)
    #训练完后，使用测试集进行测试误差
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)

#使用线性回归得到的结果是在区间[0,1]上的某个值，需要将该值转换成0或1
predictions = np.concatenate(predictions,axis=0)
#print(predictions)
#print(type(titanic["Survived"]))

predictions[predictions >.5] = 1
predictions[predictions <=.5] = 0
predictions.dtype = "float64"


titanic["Survived"] = titanic["Survived"].astype(float)

print("测试数据总数量",len(predictions))
#print("正确的数量：",sum(predictions[predictions == titanic["Survived"]]))
print("正确的数量：",sum(predictions == titanic["Survived"]))
accuracy = sum(predictions == titanic["Survived"]) / len(predictions)

print("准确率为：",accuracy)

TypeError: __init__() got an unexpected keyword argument 'n_folds'